# Clustering et word2vec

Sur la base des éléments méthodologiques et des enseignements techniques présentés lors du cours théorique, il est demandé dans le cadre de ce TP :
- d’effectuer un clustering des bulletins pour une décennie au choix et d’interpréter les résultats
- d’entraîner un modèle word2vec sur l’ensemble des bulletins et d’explorer les relations entre vecteurs

Pour ce faire, vous utiliserez différentes librairies Python vues au cours comme scikit-learn et gensim.

## 1. Clustering

## 2. Word2Vec

#### Librairies nécessaires

In [2]:
import sys
import os

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

In [3]:
sw = stopwords.words("french")

NameError: name 'stopwords' is not defined

In [40]:
with open(infile, encoding="utf-8", errors="backslashreplace") as f:
    print(type(f))

NameError: name 'infile' is not defined

In [ ]:
kept_lines = []

with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        type(f)
        content = f.readlines()
        n_lines = len(content)
        for i, line in enumerate(content):
            for w in line:
                if len(w) > 2 and w.isalph() and w.lower() not in sw:
                    kept_lines.append(" ".join(w))
        output.write(kept_lines + "\n")
print("Done")

In [ ]:
kept = []
lines = []
def clean_text(my_file):
    output_path = f"../data/sents_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(my_file, encoding='utf-8') as f:
        for line in f.read():
            for w in line:
                if len(w) > 1 and w.isalpha() and w.lower() not in sw:
                    kept.append(w)
            kept_lines = " ".join(kept)
        lines.append(kept_lines)
        output.write(lines)
    return f'Output has been written in {output_path}!'

In [ ]:
def clean_text(my_file):
    output_path = f"../data/sents_2_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(my_file, encoding='utf-8') as f:
        text = f.read()
        kword = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in kword if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string + "\n")
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(infile)

In [ ]:
cleaned_infile = f"../data/sents_2_clean.txt"

In [ ]:
text_selected = open(os.path.join(f'../data/sents_2_clean.txt'), 'r').read()
print(len(text_selected))
print(text_selected[:100])

#### Chargement des phrases

In [4]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

#### Fichier à nettoyer

In [5]:
infile = f"../data/sents_2.txt"

In [6]:
sentences = MySentences(infile)

#### Création des bigrammes

In [7]:
bigram_phrases = Phrases(sentences)

In [8]:
len(bigram_phrases.vocab.keys())

5978799

In [9]:
# exploration des bigrammes

key_ = list(bigram_phrases.vocab.keys())[145]
print(key_)

maladie


In [10]:
# score de la clé

bigram_phrases.vocab[key_]

2929

In [11]:
print(list(bigram_phrases.vocab.keys())[:10])

['v', 'i', 'v_i', 'l', 'i_l', 'l_l', 'e', 'l_e', 'de', 'e_de']


In [12]:
%time bigram_phrases[sentences]

CPU times: user 1.39 ms, sys: 10 µs, total: 1.4 ms
Wall time: 8.55 ms


In [13]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

In [14]:
%time bigram_phraser[sentences]

CPU times: user 3.09 ms, sys: 20 µs, total: 3.11 ms
Wall time: 3.86 ms


In [15]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [16]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

In [17]:
quadrigram_phrases = Phrases(trigram_phraser[sentences])

In [18]:
quadrigram_phraser = Phraser(phrases_model=quadrigram_phrases)

In [19]:
pentagram_phrases = Phrases(quadrigram_phraser[sentences])

In [20]:
pentagram_phraser = Phraser(phrases_model=pentagram_phrases)

In [21]:
corpus = list(pentagram_phraser[quadrigram_phraser[trigram_phraser[bigram_phraser[sentences]]]])

In [22]:
print(corpus[:10])

[['v', 'i', 'l', 'l', 'e', 'de', 'bruxelles', 'bulletin', 'ires', '8eanas', 'dl', '!'], ['conseil_communal', 'annee', '1847', '.'], ['au', 'ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], ['bulletin', 'conseil', 'aes', 'seances', 'communal', '.'], ['annee', '1847', '.'], ['bruxelles', ',', 'imprimerie', 'd', 'e', 'j', '.'], ['h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ',', 'rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ',', '1', '84', '8', 'de', '!'], ['du', 'consei', 'dibi', 'e', '.', '-', 'communication', 'conclusions', 'de', 'la', 'section', 'des', 'du', 'nouvel_hospice', 'pour', 'les', 'av', 'enraisonde', 'l', "'", 'absence', '&', 'maladie', '.', 'le', 'conseil', 'ajourne', 'leurs', 'de', 'pierre', 'el', 'marchai', 'cles', 'des', 'taxes', 'communale', "'", 'bieniaance', 'eldeseianv', 'il', 'est', 'donne', 'communie', ';', 'mandant', 'le', 'o', 'p', 'fa', 'gnant', 'l', "'", 'envoi', 'de', 'leur', 'bn', 'par', 'l', "'", 'e

In [23]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 2h 20min 49s, sys: 9min 36s, total: 2h 30min 25s
Wall time: 44min 14s


In [24]:
outfile = f"../data/bulletins_tp3_1.model"
model.save(outfile)

In [2]:
model = Word2Vec.load("../data/bulletins_tp3_1.model")

#### Exploration du modèle

In [5]:
model.wv.similarity("boucher", "boulanger")

0.57089794

In [30]:
model.wv.similarity("homme", "femme")

0.42860717

In [22]:
model.wv.similarity("voiture", "carrosse")

0.28735888

In [25]:
model.wv.similarity("voiture", "chien")

0.42339402

In [6]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.5818007588386536),
 ('nivelles', 0.5182838439941406),
 ('liege', 0.5148740410804749),
 ('bruxeles', 0.4937210977077484),
 ('bruxelle', 0.4917224645614624),
 ('jrvice', 0.471939355134964),
 ('xelles', 0.4709879159927368),
 ('huy', 0.47084033489227295),
 ('prague', 0.46914100646972656),
 ('binche', 0.46828752756118774)]

In [ ]:
model.wv.most_similar("boucher", topn=10)

In [27]:
model.wv.most_similar("platonisme", topn=10)

[('aristotelisme', 0.8893957734107971),
 ('symbolis', 0.8818884491920471),
 ('centum', 0.8812199831008911),
 ('christiano', 0.879470705986023),
 ('147437', 0.8793374300003052),
 ('66637', 0.8787049055099487),
 ('153125', 0.8774873614311218),
 ('gewidmet', 0.8770879507064819),
 ('43451', 0.8768373727798462),
 ('122571', 0.8763118982315063)]

In [7]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.5035298466682434), ('londres', 0.494385689496994), ('milan', 0.48436862230300903), ('tournai', 0.47917115688323975), ('huy', 0.4791364371776581), ('verviers', 0.470472127199173), ('prague', 0.4691561460494995), ('ostende', 0.46747785806655884), ('hambourg', 0.4582105278968811), ('paris', 0.4556889533996582)]


In [18]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('mere', 0.4184131324291229), ('eoi', 0.4026956558227539), ('boi', 0.39582201838493347), ('fille', 0.3846323490142822), ('famille_royale', 0.3765176236629486), ('eclale', 0.37051498889923096), ('zeepaard', 0.3684881925582886), ('sieur_van_eesbeeck', 0.36782413721084595), ('mgr', 0.3621143698692322), ('princesse_marie', 0.35621941089630127)]


In [15]:
print(model.wv.most_similar(positive=['fidelite', 'homme'], negative=['femme']))

[('obeissance', 0.538389265537262), ('<<_je_jure', 0.5057224631309509), ('je_jure_fidelite', 0.4389154016971588), ('inalterableattachement', 0.4325920641422272), ('hommage_particulier', 0.4223836660385132), ('peuple_belge', 0.42071980237960815), ('desinteressement', 0.4178043603897095), ('serment_suivant', 0.41470056772232056), ('serment', 0.4126855134963989), ('juste_hommage', 0.4065362513065338)]


In [31]:
print(model.wv.most_similar(positive=['fidelite', 'femme'], negative=['homme']))

[('<<_je_jure', 0.5098984241485596), ('obeissance', 0.5080048441886902), ('je_jure_fidelite', 0.46494928002357483), ('serment_suivant', 0.409962922334671), ('jure_fidelite', 0.4073270559310913), ('famille_royale', 0.406828373670578), ('serment_constitutionnel', 0.4066757559776306), ('respectueuse_expression', 0.4052557945251465), ('victoire', 0.3968193829059601), ('liberte_absolue', 0.3927416503429413)]


In [17]:
print(model.wv.most_similar(positive=['urinoir', 'femme'], negative=['homme']))

[('fenetre', 0.5346331000328064), ('mansarde', 0.5246878862380981), ('cloison', 0.520297110080719), ('terrasse', 0.5074345469474792), ('loge', 0.5059982538223267), ('douche', 0.49894893169403076), ('cave', 0.49847882986068726), ('chaudiere', 0.49798569083213806), ('pouponniere', 0.4923699200153351), ('laverie', 0.4922226071357727)]


In [33]:
print(model.wv.most_similar(positive=['enfant', 'homme'], negative=['femme']))

[('individu', 0.5726050138473511), ('artiste', 0.5208781361579895), ('honnete_homme', 0.5015160441398621), ('elan', 0.4846319854259491), ('jeune_homme', 0.48355039954185486), ('instrument', 0.4822431802749634), ('seul_homme', 0.47085273265838623), ('quelqu', 0.4698927104473114), ('homme_politique', 0.4673098027706146), ('ecrivain', 0.46418407559394836)]


In [34]:
print(model.wv.most_similar(positive=['enfant', 'femme'], negative=['homme']))

[('mere', 0.5782514810562134), ('fille', 0.5571849346160889), ('jeune_fille', 0.522800624370575), ('maman', 0.5204247236251831), ('servante', 0.5070370435714722), ('fillette', 0.497027724981308), ('pensionnaire', 0.4956570565700531), ('orpheline', 0.4850376546382904), ('soeur', 0.48322024941444397), ('leurs_parents', 0.48130083084106445)]


In [36]:
print(model.wv.most_similar(positive=['enfant', 'femme'], negative=['famille']))

[('individu', 0.5407490134239197), ('malade', 0.5035775303840637), ('ouvrier', 0.5015166401863098), ('fille', 0.5009419918060303), ('fillette', 0.48194992542266846), ('jeune_fille', 0.47949379682540894), ('homme', 0.47594597935676575), ('agent', 0.469593346118927), ('orphelin', 0.4688611328601837), ('indigent', 0.45601674914360046)]


In [22]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 40min 4s, sys: 1min 51s, total: 41min 56s
Wall time: 14min 27s


In [23]:
outfile = f"../data/bulletins_tp3_2.model"
model.save(outfile)

In [24]:
model = Word2Vec.load("../data/bulletins_tp3_2.model")

In [25]:
model.wv.similarity("boucher", "boulanger")

0.8609588

In [26]:
model.wv.similarity("homme", "femme")

0.62290215

In [27]:
model.wv.similarity("voiture", "carrosse")

0.41236758

In [28]:
model.wv.similarity("voiture", "chien")

0.5687413

In [29]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8323514461517334),
 ('liege', 0.7878617644309998),
 ('nivelles', 0.7507333755493164),
 ('cantonnale', 0.7505260705947876),
 ('gand', 0.7435848116874695),
 ('mons', 0.7311862111091614),
 ('paris', 0.7162811756134033),
 ('rruxelles', 0.714896559715271),
 ('flandres', 0.7096958160400391),
 ('aux_confins', 0.7072545886039734)]

In [30]:
model.wv.most_similar("boucher", topn=10)

[('batelier', 0.8890909552574158),
 ('charcutier', 0.8885161876678467),
 ('macon', 0.8633956909179688),
 ('boulanger', 0.8609588742256165),
 ('menuisier', 0.8580062389373779),
 ('marchand', 0.8434250950813293),
 ('terrassier', 0.8390104174613953),
 ('cabaretier', 0.8362985253334045),
 ('tailleur', 0.8337821960449219),
 ('poissonnier', 0.8302710652351379)]

In [31]:
model.wv.most_similar("platonisme", topn=10)

[('beveiligen', 0.9539734721183777),
 ('kundig', 0.9495147466659546),
 ('lnventaris', 0.949018120765686),
 ('onze_maatschappij', 0.9476034045219421),
 ('haare', 0.9474144577980042),
 ('jser', 0.9442475438117981),
 ('haupt', 0.9437791109085083),
 ('ontduiking', 0.941407322883606),
 ('***>', 0.9412257075309753),
 ('efdael', 0.9409153461456299)]

In [32]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('huy', 0.8370460271835327), ('ostende', 0.8271083235740662), ('verviers', 0.8080962300300598), ('gand', 0.8075035810470581), ('liege', 0.8056627511978149), ('lille', 0.7857369184494019), ('paris', 0.7746098637580872), ('louvain', 0.7729559540748596), ('flandres', 0.7717507481575012), ('alost', 0.7653114199638367)]


In [33]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('sieur_van_eesbeeck', 0.6661351323127747), ('messe_quotidienne', 0.6570274829864502), ('annep', 0.6544880867004395), ('brabant', 0.6392697095870972), ('dansante_sera', 0.6349672675132751), ('letttre', 0.6327504515647888), ('messe_solennelle', 0.6322498321533203), ('septembre_1949', 0.6276273131370544), ('congregation', 0.6272263526916504), ('unifamiliale', 0.6266288161277771)]


In [34]:
print(model.wv.most_similar(positive=['fidelite', 'homme'], negative=['femme']))

[('<<_je_jure', 0.8076946139335632), ('serment_suivant', 0.7857425808906555), ('cartel', 0.7674791812896729), ('verbe', 0.754385232925415), ('respect', 0.7543176412582397), ('mepris', 0.7468379139900208), ('je_jure_fidelite', 0.7452432513237), ('obeissance', 0.7366427183151245), ('peuple_belge', 0.7356619238853455), ('communisme', 0.7327820658683777)]


In [35]:
print(model.wv.most_similar(positive=['fidelite', 'femme'], negative=['homme']))

[('famille_royale', 0.739972710609436), ('suspension_entraine', 0.7090593576431274), ('commission_paritaire', 0.701867401599884), ('liberte_individuelle', 0.6872615814208984), ('reponse_donnee', 0.6771332025527954), ('liste_portait', 0.6767422556877136), ('elle_commettrait', 0.6667153835296631), ('prefecture', 0.6665677428245544), ('circonscription_militaire', 0.6577958464622498), ('libre_concurrence', 0.6552494168281555)]


In [36]:
print(model.wv.most_similar(positive=['urinoir', 'femme'], negative=['homme']))

[('mansarde', 0.8571958541870117), ('cave', 0.8542822003364563), ('terrasse', 0.8352386355400085), ('glaciere', 0.8255090117454529), ('baignoire', 0.8180697560310364), ('chaudiere', 0.8113610148429871), ('cloison', 0.8081401586532593), ('cloison_vitree', 0.8068118095397949), ('tremie', 0.8022738695144653), ('dalle', 0.7997750639915466)]


In [37]:
print(model.wv.most_similar(positive=['enfant', 'homme'], negative=['femme']))

[('outil', 0.8101339340209961), ('instrument', 0.7956112027168274), ('ecrivain', 0.7922137975692749), ('condisciple', 0.7866582870483398), ('interet_particulier', 0.7811781764030457), ('tremplin', 0.7791550159454346), ('elan', 0.7758635878562927), ('compagnon', 0.7740587592124939), ('desinteressement', 0.7727602124214172), ('honnete_homme', 0.767368495464325)]


In [38]:
print(model.wv.most_similar(positive=['enfant', 'femme'], negative=['homme']))

[('fillette', 0.7821374535560608), ('leur_mere', 0.776722252368927), ('fille', 0.7731388807296753), ('classe_flamande', 0.7696354985237122), ('maman', 0.768470823764801), ('carte_medicale', 0.7680832743644714), ('vaccine', 0.7643815279006958), ('incapacite', 0.7630890607833862), ('jeune_fille', 0.7610793113708496), ('servante', 0.7577486038208008)]


In [39]:
print(model.wv.most_similar(positive=['enfant', 'femme'], negative=['famille']))

[('adulte', 0.8042633533477783), ('ouvrier', 0.7701505422592163), ('homme', 0.7656218409538269), ('apprenti', 0.7634792327880859), ('enseigne', 0.7533910274505615), ('age', 0.7479268312454224), ('mendiant', 0.7372833490371704), ('externe', 0.7192428708076477), ('enfant_malade', 0.7118009924888611), ('homme_intelligent', 0.7094216346740723)]


In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

In [ ]:
outfile = f"../data/bulletins_tp3_3.model"
model.save(outfile)

In [ ]:
model = Word2Vec.load("../data/bulletins_tp3_3.model")

In [ ]:
model.wv.similarity("boucher", "boulanger")

In [ ]:
model.wv.similarity("homme", "femme")

In [ ]:
model.wv.similarity("voiture", "carrosse")

In [ ]:
model.wv.similarity("voiture", "chien")

In [ ]:
model.wv.most_similar("bruxelles", topn=10)

In [ ]:
model.wv.most_similar("boucher", topn=10)

In [ ]:
model.wv.most_similar("platonisme", topn=10)

In [ ]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

In [ ]:
print(model.wv.most_similar(positive=['fidelite', 'homme'], negative=['femme']))

In [ ]:
print(model.wv.most_similar(positive=['fidelite', 'femme'], negative=['homme']))

In [ ]:
print(model.wv.most_similar(positive=['urinoir', 'femme'], negative=['homme']))

In [ ]:
print(model.wv.most_similar(positive=['enfant', 'homme'], negative=['femme']))

In [ ]:
print(model.wv.most_similar(positive=['enfant', 'femme'], negative=['homme']))

In [ ]:
print(model.wv.most_similar(positive=['enfant', 'femme'], negative=['famille']))